# 3시간 시점

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

## 설정

In [22]:
csv_path = './dataset/must_use_final.csv'
hours_ahead = 5 # 시간 대입
interval_minutes = 10
shift_n = hours_ahead * 60 // interval_minutes
feature_cols = ['LAT', 'LON', 'COG', 'HEADING']
target_col = ['CLUSTER_1']

In [24]:
df = pd.read_csv(csv_path)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

In [26]:
accuracy_results = []

## 실행

In [28]:
for cluster_id, cluster_df in df.groupby('CLUSTER_1'):
    print(f"\n🔹 CLUSTER_1: {cluster_id}")

    # 정렬 및 미래 타깃 생성
    cluster_df = cluster_df.sort_values(by=['VSL_ID', 'TIMESTAMP']).reset_index(drop=True)
    cluster_df[target_col] = cluster_df.groupby('VSL_ID')['PORT_NAME'].shift(-shift_n)
    filtered = cluster_df.dropna(subset=feature_cols + [target_col])

    # 유효성 체크
    if len(filtered[target_col].unique()) < 2 or len(filtered) < 100:
        print("⚠️  데이터 부족 or 클래스 수 부족 → 스킵")
        continue

    # X, y 구성
    X = filtered[feature_cols].values
    y = filtered[target_col].values

    # 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # ===== Soft Voting Classifier 구성 =====
    lr = LogisticRegression(max_iter=1000)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier = VotingClassifier(estimators=[('lr', lr), ('rf', rf)], voting='soft')
    
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # 정확도 저장
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_results.append((cluster_id, accuracy))

    # 혼동 행렬 시각화
    cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))

    plt.figure(figsize=(10, 8))
    disp.plot(xticks_rotation=90)
    plt.title(f"CLUSTER_1 = {cluster_id} / Accuracy = {accuracy:.2f}")
    plt.tight_layout()
    plt.show()

# 전체 예측 결과 저장용
all_y_true = []
all_y_pred = []

# ===== CLUSTER_1 별 반복 =====
for cluster_id, cluster_df in df.groupby('CLUSTER_1'):
    ...
    if len(filtered[target_col].unique()) < 2 or len(filtered) < 100:
        ...
        continue

    ...
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # 전체 결과 누적
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    ...
    
# ===== 전체 모델 정확도 출력 =====
total_accuracy = accuracy_score(all_y_true, all_y_pred)
print(f"\n📊 전체 모델 평균 정확도: {total_accuracy:.4f}")


🔹 CLUSTER_1: 0


ValueError: Columns must be same length as key

## 최종 정확도 요약

In [38]:
print("군집별 정확도 요약")
for cid, acc in accuracy_results:
    print(f"  - CLUSTER_1 = {cid} → Accuracy = {acc:.4f}")

군집별 정확도 요약
  - CLUSTER_1 = 1 → Accuracy = 0.7479
  - CLUSTER_1 = 2 → Accuracy = 0.9180
  - CLUSTER_1 = 3 → Accuracy = 0.5248
  - CLUSTER_1 = 4 → Accuracy = 0.7768
  - CLUSTER_1 = 6 → Accuracy = 0.7990
  - CLUSTER_1 = 7 → Accuracy = 0.9675


# 과적합 & 교차검증

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [60]:
csv_path = './dataset/must_use_final.csv'
hours_ahead = 3
interval_minutes = 10
shift_n = hours_ahead * 60 // interval_minutes
feature_cols = ['LAT', 'LON', 'COG', 'HEADING']
target_col = 'FUTURE_PORT_NAME'
do_cross_validation = True  # 🔁 이걸 True로 하면 교차검증 수행

In [62]:
df = pd.read_csv(csv_path)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

In [64]:
accuracy_results = []

In [69]:
import warnings
warnings.simplefilter('ignore')

## 과적합 분석

In [71]:
# ===== CLUSTER_1 별 과적합 분석 =====
print("\n🔍 [1단계] 과적합 여부 확인")
for cluster_id, cluster_df in df.groupby('CLUSTER_1'):
    print(f"\n▶ CLUSTER_1: {cluster_id}")

    cluster_df = cluster_df.sort_values(by=['VSL_ID', 'TIMESTAMP']).reset_index(drop=True)
    cluster_df[target_col] = cluster_df.groupby('VSL_ID')['PORT_NAME'].shift(-shift_n)
    filtered = cluster_df.dropna(subset=feature_cols + [target_col])

    if len(filtered[target_col].unique()) < 2 or len(filtered) < 100:
        print("⚠️  데이터 부족 or 클래스 수 부족 → 스킵")
        continue

    X = filtered[feature_cols].values
    y = filtered[target_col].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    lr = LogisticRegression(max_iter=1000)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier = VotingClassifier(estimators=[('lr', lr), ('rf', rf)], voting='soft')

    classifier.fit(X_train, y_train)

    train_acc = classifier.score(X_train, y_train)
    test_acc = classifier.score(X_test, y_test)

    print(f"✅ 훈련 정확도: {train_acc:.4f} / 테스트 정확도: {test_acc:.4f}")
    if train_acc - test_acc > 0.1:
        print("⚠️ 과적합 가능성 있음 (훈련 정확도 >> 테스트 정확도)")


🔍 [1단계] 과적합 여부 확인

▶ CLUSTER_1: 0
⚠️  데이터 부족 or 클래스 수 부족 → 스킵

▶ CLUSTER_1: 1
✅ 훈련 정확도: 0.9785 / 테스트 정확도: 0.7933
⚠️ 과적합 가능성 있음 (훈련 정확도 >> 테스트 정확도)

▶ CLUSTER_1: 2
✅ 훈련 정확도: 0.9396 / 테스트 정확도: 0.9211

▶ CLUSTER_1: 3
✅ 훈련 정확도: 0.9979 / 테스트 정확도: 0.5354
⚠️ 과적합 가능성 있음 (훈련 정확도 >> 테스트 정확도)

▶ CLUSTER_1: 4
✅ 훈련 정확도: 0.9901 / 테스트 정확도: 0.7813
⚠️ 과적합 가능성 있음 (훈련 정확도 >> 테스트 정확도)

▶ CLUSTER_1: 5
⚠️  데이터 부족 or 클래스 수 부족 → 스킵

▶ CLUSTER_1: 6
✅ 훈련 정확도: 0.9825 / 테스트 정확도: 0.8244
⚠️ 과적합 가능성 있음 (훈련 정확도 >> 테스트 정확도)

▶ CLUSTER_1: 7
✅ 훈련 정확도: 0.9766 / 테스트 정확도: 0.9577


In [ ]:
print("\n🔁 [2단계] 5-Fold 교차검증")
for cluster_id, cluster_df in df.groupby('CLUSTER_1'):
    print(f"\n▶ CLUSTER_1: {cluster_id}")

    cluster_df = cluster_df.sort_values(by=['VSL_ID', 'TIMESTAMP']).reset_index(drop=True)
    cluster_df[target_col] = cluster_df.groupby('VSL_ID')['PORT_NAME'].shift(-shift_n)
    filtered = cluster_df.dropna(subset=feature_cols + [target_col])

    if len(filtered[target_col].unique()) < 2 or len(filtered) < 100:
        print("⚠️  데이터 부족 or 클래스 수 부족 → 스킵")
        continue

    X = filtered[feature_cols].values
    y = filtered[target_col].values

    lr = LogisticRegression(max_iter=1000)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier = VotingClassifier(estimators=[('lr', lr), ('rf', rf)], voting='soft')

    scores = cross_val_score(classifier, X, y, cv=5, scoring='accuracy')
    print(f"📊 교차검증 평균 정확도: {scores.mean():.4f} ± {scores.std():.4f}")



🔁 [2단계] 5-Fold 교차검증

▶ CLUSTER_1: 0
⚠️  데이터 부족 or 클래스 수 부족 → 스킵

▶ CLUSTER_1: 1
📊 교차검증 평균 정확도: 0.6889 ± 0.0148

▶ CLUSTER_1: 2
